In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
torch.__version__

'2.3.0+cu121'

In [4]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [9]:
# dir = '/kaggle/input/dog-emotion/Dog Emotion/'
dir='data/'

In [6]:
resnet_weights=models.ResNet50_Weights.DEFAULT

In [10]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [11]:
# train_dataset = DogEmotionDataset(
#     images=X_train.values,
#     labels=y_train.values,
#     transforms=train_transforms,
#     classes=classes
# )

# test_dataset = DogEmotionDataset(
#     images=X_test.values,
#     labels=y_test.values,
#     transforms=test_transforms,
#     classes=classes
# )

dogdataset = datasets.ImageFolder(dir, transform=resnet_weights.transforms())

train_size = int(0.9 * len(dogdataset))
test_size = len(dogdataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dogdataset, [train_size, test_size],generator=torch.Generator().manual_seed(42))

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [14]:
with open('classes.txt', 'w') as f:
    for item in dogdataset.classes:
        f.write("%s\n" % item)

In [ ]:
imgs, labels =next(iter(train_loader))
imgs.shape

In [1]:
class DogEmotionResNet(nn.Module):
    def __init__(self, num_classes, weights=None):
        super().__init__()
        
        self.resnet = models.resnet50(weights=weights)
        for param in self.resnet.parameters():
            param.requires_grad = False
        
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(in_features, num_classes)
    
    def forward(self, x):
        return self.resnet(x)

Writing model.py


In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

classes = dogdataset.classes

model = DogEmotionResNet(num_classes= len(classes), weights=resnet_weights)
model.to(device)

# test_output = model(imgs.to(device))

In [ ]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        predicted_labels = outputs.argmax(dim=1)
        correct_predictions += (predicted_labels == labels).sum().item()
        total_predictions += labels.size(0)
        
    train_loss = running_loss / len(train_loader)
    train_accuracy = correct_predictions / total_predictions
    
    return train_loss, train_accuracy

def validate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    
    with torch.inference_mode():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            predicted_labels = outputs.argmax(dim=1)
            correct_predictions += (predicted_labels == labels).sum().item()
            total_predictions += labels.size(0)
            
    test_loss = running_loss / len(test_loader)
    test_accuracy = correct_predictions / total_predictions
    
    return test_loss, test_accuracy
        

In [ ]:
from tqdm.auto import tqdm

EPOCHS = 30
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

for epoch in tqdm(range(EPOCHS)):
    train_loss, train_accuracy = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_accuracy = validate(model, test_loader, criterion, device)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)
    
    print(f'Epoch: {epoch+1}/{EPOCHS}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')
    

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='train accuracy')
plt.plot(test_accuracies, label='test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
torch.save(model.state_dict(), 'dog_emotion_model.pth')

In [ ]:
model = DogEmotionResNet(num_classes=len(classes))
model.load_state_dict(torch.load('dog_emotion_model.pth'))
model.to(device)

In [ ]:
def predict(model, test_loader, device):
    model.eval()
    predictions = []
    
    with torch.inference_mode():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            predicted_labels = outputs.argmax(dim=1)
            
            predictions.extend(predicted_labels.cpu().numpy())
            
    return predictions

predictions = predict(model, test_loader, device)

In [ ]:
predictions[:10]

In [ ]:
from sklearn.metrics import classification_report

print(classification_report([i[1] for i in test_dataset], predictions))
